### Užduotis

Pirma užduotis reikalaus realizuoti:

- [X] efektyvią duomenų nuskaitymo programą su pasirinku egzistuojančiu iš anksto apmokytu (angl. pre-trained) vaizdų klasifikavimo modeliu,
- [X] paskaičiuoti tikslumo, precizijos, atkūrimo ir F1 statistikas pasirinktiems 1000 paveikslėlių iš OpenImages,
- [X] realizuoti slenkstinės reikšmės (angl. threshold) keitimą, įgalinant klasifikuoti vaizdus kiekvienai užduotai klasei keičiant . Statistikos turi persiskaičiuoti po slenkstinės reikšmės pakeitimo.

In [1]:
%pip install -r requirements.txt

In [2]:
from pathlib import Path
from typing import Callable
from torch import Tensor
from torch.utils.data import Dataset
from PIL import Image
from importlib import reload
from copy import deepcopy
from lab1.mydataset import MyDataSet
import lab1

In [3]:
from lab1.device import device
reload(lab1.device)
device

In [4]:
from lab1.download import download_if_not_found
reload(lab1.download)
download_if_not_found()

In [12]:
from torchvision.models import resnet101, ResNet101_Weights

weights: ResNet101_Weights = ResNet101_Weights.DEFAULT

model = resnet101(weights=weights).to(device).eval()

dict([(i, c) for c, i in enumerate(weights.meta['categories'])])

In [6]:
from lab1.trans import img_trans_1

img_trans_2 = weights.transforms()
dataset = MyDataSet("data", preprocess_fn = img_trans_1, label_map = catmap)

In [13]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size = 2**4, num_workers=1)

In [14]:
from functools import reduce
import numpy as np
import torch
from torch import cat
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

predictions = torch.empty((0, 1000))
truths_ = torch.tensor([], dtype=int)

for images, ts in data_loader:
    truths_  = torch.cat((truths_, ts))
    # Can be though as list of ŷ
    #     ŷ_i - confidence that input image belongs to i-th class
    #
    #              "squishes" outputs to a range of [0, 1] 👇
    from_model = model(images.to(device)).detach().cpu().sigmoid()
    predictions = cat((predictions, from_model))

# Initializes list of vectors:
#   truths - 
#   y - ground truth vector, with only one (i-th element), representing a class set to true
truths = torch.zeros_like(predictions, dtype=torch.bool)
for index, label in enumerate(truths_):
    truths[index,label.item()] = True

truths, predictions

In [9]:
from lab1.stat import table, stats
def test():
    tp, tn, fp, fn = table(truths, predictions)
    return tp + tn + fp + fn == truths_.numel()

table(truths, predictions), test()


In [15]:
from lab1.download import classes

thr = 0.7

print(f"Threshold: {thr}")
for label in classes:
    print(f"---------------------")
    print(f"Class: {label}")
    acc, re, pre, f1 = stats(table(truths, predictions, labels=catmap[label.lower()], threshold = thr))
    print(f" . Accuracy:  {acc:.3}\n . Recall:    {re:.3}\n . Precision: {pre:.3}\n . F1:        {f1:.3}\n")

In [11]:
# It works
revcatmap = weights.meta['categories']
pred = model(
        img_trans_1(Image.open("misc/pizza.jpg")).unsqueeze(0).to(device)
    ).detach().cpu().sigmoid().squeeze().argmax()
revcatmap[pred]
